In [0]:
# root_path = Path('gdrive/My Drive/ADL/handcrafted_dataset/')
# for i in root_path.iterdir():
# classes = ['smoking','not_smoking']
# print(classes)

epochs = 10
batch_size = 32
data_root = Path('gdrive/My Drive/ADL/handcrafted_dataset/')
all_image_paths, label_names = dir_to_paths_and_labels(data_root)
a_batch = unbatched_dataset(all_image_paths, label_names, verbose=True)
a_batch = a_batch.batch(batch_size)
a_batch = a_batch.repeat(epochs)


def to_naive(x, y):
  return tf.reshape(x, shape=(-1, 12288)), y

a_batch = a_batch.map(to_naive)

b_iter = iter(a_batch)
overfit_batch = next(b_iter)
validate_batch = next(b_iter)

print(overfit_batch[0])
print(overfit_batch[1])


In [0]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

In [0]:
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from pathlib import Path

pre = Path('gdrive/My Drive/ADL')
handcrafted_root = pre/'handcrafted_dataset'
collected_root = pre/'dataset'

data_generator = ImageDataGenerator(horizontal_flip=True,
                                   width_shift_range = 0.4,
                                   height_shift_range = 0.4,
                                   zoom_range=0.3,
                                   rotation_range=20,
                                   )

image_size = 64
batch_size = 32

In [0]:
handcrafted_generator = data_generator.flow_from_directory(
        handcrafted_root,
        target_size=(image_size, image_size),
        batch_size=batch_size,
        class_mode='categorical')

generator = handcrafted_generator
num_classes = len(generator.class_indices)

dataset_iter = iter(generator)
single_batch = next(dataset_iter)
valida_batch = next(dataset_iter)

# sb = next(iter(train_generator))
# print(sb[0].reshape((-1, 64*64*3)))

In [0]:
train_generator = data_generator.flow_from_directory(
        collected_root/'train',
        target_size=(image_size, image_size),
        batch_size=batch_size,
        class_mode='categorical')

valid_generator = data_generator.flow_from_directory(
        collected_root/'validation',
        target_size=(image_size, image_size),
        batch_size=batch_size,
        class_mode='categorical')

generator = train_generator
num_classes = len(generator.class_indices)

dataset_iter = iter(generator)
single_batch = next(dataset_iter)
valida_batch = next(dataset_iter)

Found 8244 images belonging to 2 classes.
Found 1033 images belonging to 2 classes.


In [0]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

model = resnet50_model()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

# model = naive_model(hidden=128)
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) 

# single_batch = next(iter(train_generator))
# model.fit(single_batch[0].reshape(-1, (image_size*image_size*3)), single_batch[1], steps_per_epoch=5, epochs=30)
# int(count/batch_size) + 1,
#        batch_size=batch_size,

In [0]:
model.fit(
    single_batch[0],
    single_batch[1],
    steps_per_epoch=5,
    validation_data=valida_batch,
    validation_steps=1,
    verbose = 1,
    epochs=60)

In [0]:
model.fit_generator(
    train_generator,
    validation_data=valida_batch,
    validation_steps=1,
    verbose = 1,
    epochs=60)

Epoch 1/60
258/258 [==============================] - 2676s 10s/step - loss: 0.9233 - binary_accuracy: 0.6452 - precision_1: 0.6452 - recall_1: 0.6452 - val_loss: 0.4953 - val_binary_accuracy: 0.8438 - val_precision_1: 0.8438 - val_recall_1: 0.8438
Epoch 2/60
258/258 [==============================] - 49s 190ms/step - loss: 0.5983 - binary_accuracy: 0.6886 - precision_1: 0.6886 - recall_1: 0.6886 - val_loss: 0.5296 - val_binary_accuracy: 0.7188 - val_precision_1: 0.7188 - val_recall_1: 0.7188
Epoch 3/60
258/258 [==============================] - 49s 189ms/step - loss: 0.5937 - binary_accuracy: 0.6919 - precision_1: 0.6919 - recall_1: 0.6919 - val_loss: 0.4982 - val_binary_accuracy: 0.7500 - val_precision_1: 0.7500 - val_recall_1: 0.7500
Epoch 4/60
258/258 [==============================] - 49s 191ms/step - loss: 0.5874 - binary_accuracy: 0.6951 - precision_1: 0.6951 - recall_1: 0.6951 - val_loss: 0.5118 - val_binary_accuracy: 0.7188 - val_precision_1: 0.7188 - val_recall_1: 0.7188
Epoc

In [0]:
model = naive_model() # keras_model()

model.compile(
    optimizer=tf.train.AdamOptimizer(), 
    loss=tf.keras.losses.binary_crossentropy,
    metrics=['accuracy', 'binary_crossentropy']) # tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), 


model.fit(
    overfit_batch[0],
    overfit_batch[1],
    steps_per_epoch=len(overfit_batch[0]) // batch_size,
    epochs=epochs,
    validation_data=validate_batch,
    validation_steps=1,
    verbose = 1)


In [0]:
from __future__ import absolute_import, division, print_function


from pathlib import Path
import random

def dataset(data_root, batch_size, verbose=False, autotune=tf.data.experimental.AUTOTUNE):
    all_image_paths, label_names = dir_to_paths_and_labels(data_root)
    unbatched_ds = unbatched_dataset(all_image_paths, label_names, verbose=verbose)

    ds = unbatched_ds.apply(tf.data.experimental.shuffle_and_repeat(buffer_size=len(all_image_paths)))
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=autotune)
    return ds, label_names, all_image_paths

def all_dataset(data_root, verbose=False):
    all_image_paths, label_names = dir_to_paths_and_labels(data_root)
    unbatched_ds = unbatched_dataset(all_image_paths, label_names, verbose=verbose)
    return unbatched_ds, label_names, all_image_paths

def unbatched_dataset(all_image_paths, label_names, verbose=False, limit=None):
    label_to_index = dict((name, index) for index,name in enumerate(label_names))
    all_image_labels = [label_to_index[Path(path).parent.name] for path in all_image_paths]

    verbose_size = 10
    if (limit):
      all_image_paths  = all_image_paths[:limit]
      all_image_labels = all_image_labels[:limit]
      verbose_size = limit

    if verbose:
        print("label to index map ", label_to_index)
        print(f"First {verbose_size} labels indices: {all_image_labels[:verbose_size]}")
        print(f"First {verbose_size} paths indices: {all_image_paths[:verbose_size]}")

    ds = tf.data.Dataset.from_tensor_slices((all_image_paths, all_image_labels))      
    image_label_ds = ds.map(load_and_preprocess_from_path_label)
    return image_label_ds

def dir_to_paths_and_labels(data_root):
    all_image_paths = dir_to_files(data_root)
    label_names = sorted(item.name for item in data_root.glob('*/') if item.is_dir())
    return all_image_paths, label_names

def dir_to_files(data_root):
    all_image_paths = list(data_root.glob('*/*'))
    all_image_paths = [str(path) for path in all_image_paths if path.suffix != '']
    random.shuffle(all_image_paths)
    return all_image_paths

#
# If imagenet weights are being loaded, input must have a static square 
#    shape(one of (96, 96), (128, 128), (160, 160),(192, 192), or (224, 224)
def preprocess_image(image, sizes=(64, 3)):
  image = tf.image.decode_jpeg(image, channels=sizes[1])
  image = tf.image.resize_images(image, [sizes[0], sizes[0]])
  image /= 255.0  # normalize to [0,1] range
  image = tf.reshape(image, (1, sizes[0], sizes[0], sizes[1]))
  return image

def load_and_preprocess_image(path):
  print(f"{path}")
  image = tf.read_file(path)
  return preprocess_image(image)

def load_and_preprocess_from_path_label(path, label):
  return load_and_preprocess_image(path), tf.one_hot(tf.cast(label, tf.uint8), 2) # label # 

In [0]:
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, Input, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.python.keras.applications import ResNet50

def naive_model(input_shape=(64, 64, 3), hidden=12288, num_classes=2):
  model = Sequential()
  model.add(Dense(hidden, activation=None))
#  model.add(Dense(4, activation=tf.nn.relu))
  model.add(Dense(num_classes, activation=tf.nn.sigmoid))
  return model


def resnet50_model(num_classes=2):
  model = Sequential()
  #keras.applications.resnet.ResNet50(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)
  model.add(ResNet50(include_top=False, pooling='avg', weights='imagenet'))
  model.add(Flatten())
  model.add(BatchNormalization())
  model.add(Dense(2048, activation='relu'))
  model.add(BatchNormalization())
  model.add(Dense(1024, activation='relu'))
  model.add(BatchNormalization())
  model.add(Dense(num_classes, activation='softmax'))
  model.layers[0].trainable = False
  return model


def keras_model(input_shape=(64, 64, 3), num_classes=2):
    model = Sequential()
    if len(input_shape) < 3:
        model.add(Lambda(lambda x: tf.expand_dims(x, -1), input_shape=input_shape))
        input_shape = (input_shape[0], input_shape[1], 1)
    model.add(Conv2D(32, (3, 3), activation='relu', padding='valid', input_shape=input_shape))
    model.add(MaxPool2D(pool_size=(2, 2)))
#    model.add(Dropout(0.2))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    return model
#     inputs = Input(shape=input_shape)
#     x = Conv2D(32, (3, 3),activation='relu', padding='valid')(inputs)
#     x = MaxPool2D(pool_size=(2, 2))(x)
#     x = Conv2D(64, (3, 3), activation='relu')(x)
#     x = MaxPool2D(pool_size=(2, 2))(x)
#     x = Flatten()(x)
#     x = Dense(512, activation='relu')(x)
#     x = Dropout(0.5)(x)
#     outputs = Dense(num_classes, activation='softmax')(x)    
#     return tf.keras.Model(inputs, outputs)

In [0]:
import IPython.display as display

def caption_image(image_path):
    image_rel = Path(image_path).relative_to(data_root)
    return "Image " + str(image_rel)
  
for n in range(10):
  image_path = random.choice(all_image_paths)
  display.display(display.Image(image_path))
  print(caption_image(image_path))
  print()


  

In [0]:
import tensorflow as tf
tf.enable_eager_execution()
print(f"Tensorflow version: {tf.__version__}")

if tf.test.is_gpu_available():
  print('Using GPU')
else:
  print('Not using GPU')
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Tensorflow version: 1.13.1
Using GPU
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
